In [51]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties


# > 85% match
sza_thres           = [70, 75]    # degrees, Tropomi filters SZA > 75. 
albedo_thres        = [0.02, 0.06]# 
elev_thres          = [80, 100]   # meters, tropomi 
cs_thres            = [0.2, 0.3]  # reducing from 0.65 because it masks out too many regions  0 (cloudy), 1 (clear)
wind_thres          = [4, 10]     # ms-1 

difficult_threshold = 7
medium_treshold     = 1

fnameout_root = f"diff{difficult_threshold}_med{medium_treshold}_sza{sza_thres[0]}_{sza_thres[1]}_elev{elev_thres[0]}_{elev_thres[1]}_csThres{cs_thres[0]}_{cs_thres[1]}_albedoSwir{albedo_thres[0]}_{albedo_thres[1]}_wind{wind_thres[0]}_{wind_thres[1]}"

In [52]:
mines = pd.read_csv(
    r'C:\Users\rebek\Ember\Data Tool\cmm-data-tool\query_result_2025-12-01T11_49_18.284701141Z.csv', # Reads in data from gem_mines_raw from metabase
    thousands=",",       # interpret commas as thousand separators inside numbers
    quotechar='"',       # respect quoted fields
    engine="python"      # more flexible parser for tricky CSVs
)

mines["IS_LIGNITE"] = mines["COAL_TYPE"].isin(
    ["Lignite", "Subbituminous / Lignite"]
)

mines = mines[mines['IS_LIGNITE'] == False]

In [53]:
china_prod = pd.read_csv(
    r'C:\Users\rebek\Ember\Data Tool\cmm-data-tool\data_historical\assets\production\Global-Coal-Mine-Tracker-September-2024-Supplement-v2_china.csv',
    encoding='latin1',      # or 'ISO-8859-1'
    thousands=',',
    quotechar='"',
    engine='python'
)

# Keep only rows where 'Mine Name' is in the list of mine names
china_prod_filtered = china_prod[china_prod['GEM Mine ID'].isin(mines['GEM_MINE_ID'])]
# china_prod_filtered = china_prod[china_prod['Mine Name'].isin(mines['MINE_NAME'])]

# Optional: reset index after dropping rows
china_prod_filtered = china_prod_filtered.reset_index(drop=True)

In [54]:
# Read the non-China CSV
non_china_prod = pd.read_csv(
    r'C:\Users\rebek\Ember\Data Tool\cmm-data-tool\data_historical\assets\production\Global-Coal-Mine-Tracker-September-2024-Supplement-v2_non_china.csv',
    encoding='latin1',
    thousands=',',
    quotechar='"',
    engine='python'
)

non_china_prod.rename(columns={'ï»¿GEM Mine ID': 'GEM Mine ID'}, inplace=True) 

# Original filter: GEM Mine ID is in mines['GEM_MINE_ID']
mask_in_mines = non_china_prod['GEM Mine ID'].isin(mines['GEM_MINE_ID'])

# Rows where Lignite (Manual) is False
mask_not_lignite = non_china_prod['Lignite (Manual)'] == 'FALSE'

# Combine both conditions with OR
non_china_prod_filtered = non_china_prod[mask_in_mines | mask_not_lignite]

# Reset index
non_china_prod_filtered = non_china_prod_filtered.reset_index(drop=True)

In [55]:

non_china_prod_aligned = non_china_prod_filtered[china_prod_filtered.columns.intersection(non_china_prod_filtered.columns)]

historical_prod = pd.concat([china_prod_filtered, non_china_prod_aligned], ignore_index=True)

print(historical_prod.shape)

output_columns = [
    'Coal Output (Annual, Mt) 2023', 
    'Coal Output (Annual, Mt) 2022', 
    'Coal Output (Annual, Mt) 2021', 
    'Coal Output (Annual, Mt) 2020', 
    'Coal Output (Annual, Mt) 2019', 
    'Coal Output (Annual, Mt) 2018', 
    'Coal Output (Annual, Mt) 2017'
]


# get latest value to align with Ioannis's oil/gas method
historical_prod['Most Recent Coal Output (Annual, Mt)'] = historical_prod[output_columns].bfill(axis=1).iloc[:, 0]

# Step 0: Replace '-' with NaN in Most Recent Coal Output
historical_prod['Most Recent Coal Output (Annual, Mt)'] = historical_prod['Most Recent Coal Output (Annual, Mt)'].replace('-', np.nan)

# Step 1: Create a mapping from MINE_NAME to the last PRODUCTION__MTPA value
mine_prod_map = mines.groupby('MINE_NAME')['PRODUCTION__MTPA'].last()

# Step 2: Fill missing values in Most Recent Coal Output by looking up Mine Name
historical_prod['Most Recent Coal Output (Annual, Mt)'] = historical_prod['Most Recent Coal Output (Annual, Mt)'].fillna(
    historical_prod['Mine Name'].map(mine_prod_map)
)

historical_prod.rename(columns={'GEM Mine ID': 'Unit ID'}, inplace=True) 

historical_prod.to_csv('historical_prod_most_recent.csv', index=False)


(1696, 24)


In [56]:
from scipy.stats import mode

# get monthly scores at location of coal mines. from vis_score_3classes_alternative.ipynb
fname = r'C:\Users\rebek\Ember\Data Tool\cmm-data-tool\data_historical\combined_score\\' + fnameout_root + 'combined_classes.csv'

combined_scores = pd.read_csv(fname,na_values='--')

combined_scores = combined_scores[combined_scores['Status'] == 'Operating'] 

coal_scores = combined_scores[combined_scores['Fuel type'] == 'coal'] 

coal_scores = coal_scores.copy()

# Function to calculate mode for a given row
def calculate_mode(row):
    return mode(row, keepdims=True).mode[0]

# extract monthly scores
subset_coal = coal_scores.iloc[:, -12:]

# get mode from monthly data
coal_scores['Mode'] = subset_coal.apply(calculate_mode, axis=1)

coal_scores.loc[:,'year_frac_0'] = subset_coal.apply(lambda row: sum(1 for value in row if value == 0 ) / 12, axis=1)
coal_scores.loc[:,'year_frac_1'] = subset_coal.apply(lambda row: sum(1 for value in row if value == 1 ) / 12, axis=1)
coal_scores.loc[:,'year_frac_2'] = subset_coal.apply(lambda row: sum(1 for value in row if value == 2 ) / 12, axis=1)

# use production values from the supplement 2024
merged_df = pd.merge(coal_scores, historical_prod[['Unit ID', 'Most Recent Coal Output (Annual, Mt)']], on='Unit ID', how='right')

# merge the scores and the production data
merged_df['has_production'] = merged_df['Most Recent Coal Output (Annual, Mt)'].notna() & (merged_df['Most Recent Coal Output (Annual, Mt)'] != '')

# drop production data if there is no score. 
merged_df = merged_df.dropna(subset=['Mode'])

# drop if there is no production data
merged_df = merged_df[merged_df['has_production']]

# Convert production column to numeric
merged_df['Most Recent Coal Output (Annual, Mt)'] = pd.to_numeric(
    merged_df['Most Recent Coal Output (Annual, Mt)'], errors='coerce'
)

# Drop rows where conversion failed (if any)
merged_df = merged_df.dropna(subset=['Most Recent Coal Output (Annual, Mt)'])

# multiply production by fraction of year with 0,1,2
merged_df['year_frac_0_prod'] = merged_df['year_frac_0']*merged_df['Most Recent Coal Output (Annual, Mt)']
merged_df['year_frac_1_prod'] = merged_df['year_frac_1']*merged_df['Most Recent Coal Output (Annual, Mt)']
merged_df['year_frac_2_prod'] = merged_df['year_frac_2']*merged_df['Most Recent Coal Output (Annual, Mt)']

# get production for based on fraction of the year with 0,1,2
production_year_frac_0 = merged_df.groupby('Country', as_index=False)['year_frac_0_prod'].sum()
production_year_frac_1 = merged_df.groupby('Country', as_index=False)['year_frac_1_prod'].sum()
production_year_frac_2 = merged_df.groupby('Country', as_index=False)['year_frac_2_prod'].sum()


final_result = pd.merge(production_year_frac_0, production_year_frac_1, on='Country', how='outer' )
final_result = pd.merge(final_result, production_year_frac_2, on='Country', how='outer')

final_result['production_year_frac_0_1_sum'] = final_result[['year_frac_0_prod', 'year_frac_1_prod']].sum(axis=1, skipna=True)

final_result_sorted = final_result.sort_values(by='production_year_frac_0_1_sum', ascending=False).reset_index(drop=True)

# Compute world totals (sum across all countries)
world_row = final_result_sorted[['year_frac_0_prod', 'year_frac_1_prod', 'year_frac_2_prod',
                                 'production_year_frac_0_1_sum']].sum()

# Turn into a DataFrame row
world_row = world_row.to_frame().T
world_row['Country'] = 'World'

# Match column order
world_row = world_row[final_result_sorted.columns]

# Append to the bottom
final_result_sorted = pd.concat(
    [final_result_sorted, world_row],
    ignore_index=True
)

# year_frac_0_prod = Production in difficult category
# year_frac_1_prod = Production in moderate category
# year_frac_2_prod = Production in favourable category

In [58]:
output_data = pd.DataFrame()
output_data['Country'] = final_result_sorted['Country']
output_data['% Coal Production with Favourable Conditions for Satellite Methane Monitoring'] = 100*(final_result_sorted['year_frac_2_prod'] / (final_result_sorted['year_frac_0_prod'] + final_result_sorted['year_frac_1_prod'] + final_result_sorted['year_frac_2_prod']))

output_data.to_csv('gem_coal_production_per_country.csv', index=False)